In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import pandas as pd

In [2]:
# Hyper Parameters 
num_classes = 400
num_epochs = 5
batch_size = 64

In [3]:
train_path = "/kaggle/input/csc4851-homework4/birds_400/train"
val_path = "/kaggle/input/csc4851-homework4/birds_400/valid"
test_path = "/kaggle/input/csc4851-homework4/birds_400/test" 

transform = transforms.Compose([transforms.Resize(64),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_image = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
valid_image = torchvision.datasets.ImageFolder(root=val_path, transform=transform)
test_image = torchvision.datasets.ImageFolder(root=test_path, transform=transform)

train_ds = DataLoader(train_image, batch_size=batch_size)
valid_ds = DataLoader(valid_image, batch_size=batch_size)
test_ds = DataLoader(test_image, batch_size=batch_size)

In [5]:
class Bird(nn.Module):
    
    def __init__(self, num_classes):
        super(Bird, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
num_classes = len(train_image.classes)
model = Bird(num_classes)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [8]:
train_steps = len(train_ds)
valid_steps = len(valid_ds)
metrics = {"loss": [], "val_loss": [], "val_accuracy": []}

for epoch in range(num_epochs):
    train_losses = []
    valid_losses = []
    model.train()
    begin = time.time()
    for batch in train_ds:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.data.item())
        if len(train_losses) > 0 and len(train_losses) % 20 == 0:
            current = time.time()
            elapsed = current - begin
            print("Epoch %d: [Training] %.2fs/%.2fs"%(epoch + 1, elapsed, elapsed / float(len(train_losses)) * train_steps))
    model.eval()
    num_correct = 0
    num_samples = 0
    begin = time.time()
    for batch in valid_ds:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        loss = loss_fn(output, targets)
        valid_losses.append(loss.data.item())
        correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
        num_correct += torch.sum(correct).item()
        num_samples += correct.shape[0]
        if len(valid_losses) > 0 and len(valid_losses) % 10 == 0:
            current = time.time()
            elapsed = current - begin
            print("Epoch %d: [Validation] %.2fs/%.2fs"%(epoch + 1, elapsed, elapsed / float(len(valid_losses)) * valid_steps))
    train_loss = torch.mean(torch.Tensor(train_losses)).item()
    valid_loss = torch.mean(torch.Tensor(valid_losses)).item()
    accuracy = num_correct / num_samples if num_samples > 0 else 0
    metrics["loss"].append(train_loss)
    metrics["val_loss"].append(valid_loss)
    metrics["val_accuracy"].append(accuracy)
    print("Training Loss: %.2f Validation Loss: %.2f accuracy: %.2f" %(train_loss, valid_loss, accuracy))

In [9]:
pd.DataFrame(metrics).plot()